In [60]:

# Import the required modules.
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from sklearn.svm import SVC
import nltk
from nltk.stem import PorterStemmer
import re

In [61]:

# Load the dataset.
data= scipy.io.loadmat('spamTrain.mat')

In [62]:

data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [63]:

data["X"].shape

(4000, 1899)

In [64]:

# Initialize the feature vector generated by the processing of the input mail.
featur_vector=np.zeros((1899,1))

In [65]:

# Processing the email into a vector related to a vocabulary of the top 1899 words found in spam emails by making the required transformations
def process_the_email(email,vector):
    def remove_html_tags(text):
        # Define the regular expression pattern to match an HTML tag
        clean = re.compile('<.*?>')
        # Use re.sub() to replace an HTML tag with a space
        return re.sub(clean, '', text)

    def replace_links_with_httpaddr(text):
        # Define the regular expression pattern to match URLs
         url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
         # Use re.sub() to replace URLs with "httpaddr"
         modified_text = re.sub(url_pattern, 'httpaddr', text)
    
         return modified_text

    def replace_email_address_with_emailaddr(text):
        # Define the regular expression pattern to match email address
        pattern=r'\b[\w\.-]+@[\w\.-]+\.\w+\b'
        # Use re.sub() to replace email address with "emailaddr"
        modified_text = re.sub(pattern, 'emailaddr', text)
    
        return modified_text
    def replace_number_with_number(text):
         # Define the regular expression pattern to match any number.
        pattern = r'\b\d+\b'
        # Use re.sub() to replace numbers with "number"
        modified_text = re.sub(pattern, 'number', text)
    
        return modified_text

    def replace_dollar_sign_with_dollar(text):
        # Define the regular expression pattern to match a dollar sign
        pattern = r'\$'
        # Use re.sub() to replace $ with "dollar"
        modified_text = re.sub(pattern, 'dollar', text)
    
        return modified_text
    def process_text(text):
        # Remove punctuation using regular expressions
        text = re.sub(r'[^\w\s]', '', text)
    
        # Replace multiple white spaces (including tabs and newlines) with a single space
        text = re.sub(r'\s+', ' ', text)
     
        return text  
    # Created a function that converts the email into a feature vector that can be input into the spam classifier we trained 
    def featur_vectorisation(vector,text):
        with open('vocab.txt', 'r') as file:# 'vocab.txt' is the word list we are using to create the feature vector
             for lines in file:
                    a=lines.split() # The vector has the same number of elements as the list. A '0' in a position of the vector reflects the absence of the same word, which is indexed 1 less than the index of the vector. Likewise, the presence is represented by '1'
                    if a[1] == text:
                        vector[int(a[0])-1]=1
                    else:
                        vector[int(a[0])-1]=vector[int(a[0])-1]+0       
   
        return vector
    
    
    stemmer = PorterStemmer()
    
    with open(email,'r') as file:
         for line in file:
                line=line.lower()
                cleaned_text = remove_html_tags(line)
                cleaned_text=replace_links_with_httpaddr(cleaned_text)
                cleaned_text=replace_email_address_with_emailaddr(cleaned_text)
                cleaned_text=replace_number_with_number(cleaned_text)
                cleaned_text=replace_dollar_sign_with_dollar(cleaned_text)
                cleaned_text=process_text(cleaned_text)
                for word in cleaned_text.split():
                    word=stemmer.stem(word) 
                    vector=featur_vectorisation(vector,word)
                    
    return vector

In [66]:

vector=process_the_email('spamSample1.txt',featur_vector)
count_ones = np.count_nonzero(vector == 1) # A spam email is converted into its feature vector.
print("Number of 1s:", count_ones)


Number of 1s: 46


In [67]:

vector.shape

(1899, 1)

In [68]:

data["X"].shape

(4000, 1899)

In [69]:

svm_classifier = SVC(kernel='rbf') # Training the model with the data 
model=svm_classifier.fit(data['X'],data['y'])

C:\Users\devas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [70]:

score_on_train_set=model.score(data['X'], data['y']) # Calculating the performance

In [71]:

score_on_train_set

0.99325

In [72]:

data_test= scipy.io.loadmat('spamTest.mat')

In [73]:

data_test.keys()


dict_keys(['__header__', '__version__', '__globals__', 'Xtest', 'ytest'])

In [74]:

score_on_test_set=model.score(data_test['Xtest'], data_test['ytest']) #Calculating the performance on the test set 

In [75]:

score_on_test_set

0.987

In [76]:

prediction=model.predict(vector.T) #Predicting whether the email is spam or not using the model
if prediction == 1:
    print('the mail is spam')
else:
    print('the mail is notspam')

the mail is spam
